In [4]:
# Importar librerías 

import pandas as pd  

from bs4 import BeautifulSoup
import requests

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
from time import sleep  

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.




In [4]:
class ExtraerDatosPeliculasSelenium:
    def __init__(self, archivo_csv): 
        self.archivo_csv = pd.read_csv(archivo_csv)  
        
        self.info_peliculas= {"puntuacion_imdb": [],
                            "puntuacion_rotten_tomatoes":[],
                            "Director":[],
                            "Guionista":[],
                            "Argumento": [],
                            "Duración":[],
                            "Nombre de la película":[],
                            "Id pelicula":[]} 
        self.info_actores = {"Nombre" : [],
                             "Año Nacimiento" : [],
                             "Peliculas Realizadas" : [],
                             "Que hace" : [],
                             "Premios" : []}
    

    def pag_principal_google(self):
        # Inicializar el driver de Chrome
        self.driver = webdriver.Chrome()
        sleep(2)
        self.driver.maximize_window()
        self.driver.get("https://www.google.es/")
        # acepta cookies
        sleep(10)
        try:
            self.driver.find_element("css selector", "#L2AGLb").click()
        except:
            pass
        self.escoger_pelicula_csv()         

    def escoger_pelicula_csv (self):        
        # Itera por cada pelicula del csv
        for peli in range(0, len(self.archivo_csv)):
            titulo_a_buscar = self.archivo_csv['Titulo'][peli]
            anho_a_buscar = self.archivo_csv['Año'][peli]
            print(titulo_a_buscar,anho_a_buscar )
            self.info_peliculas["Id pelicula"].append(self.archivo_csv['Id'][peli])
            self.info_peliculas["Nombre de la película"].append(titulo_a_buscar)         
            self.busqueda_google_imdb(titulo_a_buscar, anho_a_buscar)
        # self.buscar_info_actores()
        
    def busqueda_google_imdb(self, titulo_a_buscar, anho_a_buscar):
        
        sleep(5)
        # Ingresar el título y año de la película en el campo de búsqueda
        self.driver.find_element("css selector", "#APjFqb").clear()
        self.driver.find_element("css selector", "#APjFqb").send_keys(f'imdb "{titulo_a_buscar} {anho_a_buscar}"', Keys.ENTER)
        sleep(5)        
        # Entra en el primer resultado de google        
        try:
            # href = print(self.driver.find_element("css selector", "#rso .yuRUbf > div > span > a").click
            self.driver.find_element("css selector", "#rso .yuRUbf > div > span > a").click()
            self.entrar_coger_info_peliculas_imdb()
            self.busqueda_google_tomatoes(titulo_a_buscar, anho_a_buscar)
            print(self.info_peliculas)        
        except:
            print(f"la peli {titulo_a_buscar} {anho_a_buscar} no aparece en la primera posicion del buscador")
                
        sleep(5)     
    
        
    def entrar_coger_info_peliculas_imdb(self):
        # Acepta cookies 
        try:
            self.driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        except:
            pass

    #  coger_puntuacion
        selector_puntuacion = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-3a4309f8-0.bjXIAP.sc-1f50b7c-1.bfrqUc > div > div:nth-child(1) > a > span > div > div.sc-eb51e184-0.kFvAju > div.sc-eb51e184-2.czkfBq > span.sc-eb51e184-1.cxhhrI"
        try:
            sleep(10)
            puntuacion = self.driver.find_elements("css selector", selector_puntuacion).text
            self.info_peliculas["puntuacion_imdb"].append(puntuacion)
            
        except:       
            self.info_peliculas["puntuacion_imdb"].append("None")

        # Coger directores            
        selector_directores = '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/ul/li[1]/div'
        try:    
            sleep(10)
            self.info_peliculas["Director"].append(self.driver.find_element("xpath", selector_directores).text)            
        except:        
            print(f"Error al sacar guionistas")
            self.info_peliculas["Director"].append("None")
        
        # Coger guionistas
        selector_guionista = '#__next .sc-491663c0-10.rbXFE .sc-1f50b7c-3.ZYFjc > div > ul > li:nth-child(2) > div > ul'
       
        sleep(10)
        try:                
            self.info_peliculas["Guionista"].append(self.driver.find_element("xpath",selector_guionista ).text)
        except:       
            self.info_peliculas["Guionista"].append("None")

         # Coger argumento
        selector_argumento = '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p > span.sc-2d37a7c7-2.PeLXr'
        try:
            sleep(10)
            self.info_peliculas["Argumento"].append(self.driver.find_element("css selector", selector_argumento).text)
            
        except:
            self.info_peliculas["Argumento"].append("None")

        # Coger duracion pelicula
        selector_duracion = '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]'
        try:
            sleep(10)
            self.info_peliculas["Duración"].append(self.driver.find_element("xpath", selector_duracion).text)
        except:
            self.info_peliculas["Duración"].append("None")
               
        # Coger actores
       
        for i in range(1,11):   
            try:
                sleep(10)
                self.info_actores["Nombre"].append(self.driver.find_element("xpath", f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]/div[{i}]/div[2]/a').text)       
                           
            except:
                print(f"Error al sacar actores {i}")
        
        self.driver.back()

    def busqueda_google_tomatoes(self,titulo_a_buscar, anho_a_buscar):  
        sleep(3)
        self.driver.find_element("css selector", "#APjFqb").clear()        
        # Ingresar el título y año de la película en el campo de búsqueda
        sleep(2)
        self.driver.find_element("css selector", "#APjFqb").send_keys(f'Rotten Tomatoes Puntuación {titulo_a_buscar} {anho_a_buscar}', Keys.ENTER)
        sleep(5)
    
        # Entrar en la url            
        try:
            sleep(5)
            self.driver.find_element("xpath", '#rso > div:nth-child(1) > div > div > div > div.kb0PBd.cvP2Ce.A9Y9g.jGGQ5e > div > div > span > a').click()
            sleep(5)            
            # Aceptar cookies
            try:
                sleep(5)
                self.driver.find_element("xpath", '//*[@id="onetrust-accept-btn-handler"]').click()
            except:
                pass
            try:
                sleep(5)                
                puntuacion_tomatoes = self.driver.find_element("xpath", '//*[@id="celebrity"]/article/section[3]/div/div/table/tbody/tr[1]/td[1]/span/span[2]').text
                self.info_peliculas["puntuacion_rotten_tomatoes"].append(puntuacion_tomatoes)
            except:
                self.info_peliculas["puntuacion_rotten_tomatoes"].append("None")     
        except:
            print("No consigo entrar en Tomatoes")
        self.driver.back()


    def buscar_info_actores (self):
        # introduce actores en buscador
        sleep(3)
        
        for i in range(0, len(self.info_actores["Nombre"])):
            try:
                self.buscador = self.driver.find_element("css selector", "#APjFqb")
                self.buscador.clear()
                sleep(5)
                self.buscador.send_keys(f'imdb "{self.info_actores["Nombre"][i]}"', Keys.ENTER)   
                try:
                    sleep(10)
                    self.driver.find_element("css selector", "#rso > div.hlcw0c > div > div > div > div > div > div > div > div.yuRUbf > div > span > a").click()
                    self.coger_datos_actores(self)
                    sleep(2)
                except:
                    print(f"El actor {self.lista_actores[i]} no aparece en la primera posicion del buscador")
                        
            except:
                pass   
            
    
    def coger_datos_actores(self):
        # Acepta cookies 
        try:
            sleep(5)
            self.driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
            sleep(2)
        except:
            pass
        # Coger info actores  
        print("coger info actores")      
        try:
            sleep(2)
            anho_nacimiento = self.driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > section > aside > div:nth-child(1) > span:nth-child(2)').text
            self.info_actores["Año nacimeinto"].append(anho_nacimiento)
        except:
            self.info_actores["Año nacimeinto"].append("None")
    
        # Coger peliculas del actores
        try:
            sleep(2)
            pelis_realizadas = self.driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1)').text
            self.info_actores["Peliculas Realizadas"].append(pelis_realizadas)
        except:
            self.info_actores["Peliculas Realizadas"].append("None")

        # Coger que hace el actores
        try:
            sleep(2)
            que_hace_actor = self.driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div > ul').text
            self.info_actores["Que haces"].append(que_hace_actor)
        except:
            self.info_actores["Que hace"].append("None")
        

        # Coger premios
        try:
            self.driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > a.ipc-metadata-list-item__icon-link').text
            sleep(2)        
            for i in range(2,7):
            # Comprobar si es nominado o ganado
                try:
                    eval_premio = self.driver.find_element("css selector", f'#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child({i}) > div.sc-f65f65be-0.bBlII > ul > li > div.ipc-metadata-list-summary-item__c > div > a').text
                    if "Ganad" in eval_premio:
                        try:
                            premios = self.driver.find_element("css selector", f'#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child({i})').text
                            self.info_actores["Premios"].append(premios)
                        except:
                            pass
                except:
                    self.info_actores["Que Premios"].append("None")
        except:
            print("No entra en premios")

        self.driver.back()
        print(self.info_peliculas) 
        

In [5]:
csv = "Action.csv"
peliculas_accion_sele = ExtraerDatosPeliculasSelenium(csv)
peli = peliculas_accion_sele.pag_principal_google()


Tekken 2010
No consigo entrar en Tomatoes
{'puntuacion_imdb': ['None'], 'puntuacion_rotten_tomatoes': [], 'Director': ['Dwight H. Little'], 'Guionista': ['None'], 'Argumento': ['Jin Kazama es testigo de la muerte de su madre Jun por parte de Tekken.'], 'Duración': ['1h 31min'], 'Nombre de la película': ['Tekken'], 'Id pelicula': ['tt0411951']}
Venus & Vegas 2010
No consigo entrar en Tomatoes
{'puntuacion_imdb': ['None', 'None'], 'puntuacion_rotten_tomatoes': [], 'Director': ['Dwight H. Little', 'Demian Lichtenstein'], 'Guionista': ['None', 'None'], 'Argumento': ['Jin Kazama es testigo de la muerte de su madre Jun por parte de Tekken.', 'When three Vegas buddies attempt the score of a lifetime, they have to walk a fine line between their girlfriends who want their heads, and the mobsters who want them dead.'], 'Duración': ['1h 31min', '1h 35min'], 'Nombre de la película': ['Tekken', 'Venus & Vegas'], 'Id pelicula': ['tt0411951', 'tt0423474']}
The A-Team 2010
Error al sacar guionistas


In [7]:
peliculas_accion_sele.info_actores

NameError: name 'peliculas_accion_sele' is not defined

: 

In [6]:
import requests

# Definimos la url de la página de la que vamos a sacar datos
url_imbd = "https://www.imdb.com/title/tt0411951/"

# Definimos un User-Agent para que la solicitud parezca venir de un navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Hacemos la request a la página de la que queremos sacar la info
res_moviles = requests.get(url_imbd, headers=headers)

# Vemos si todo ha ido bien
print("La respuesta de la petición es:", res_moviles.status_code)


La respuesta de la petición es: 200


In [7]:
# creamos el objeto BeautifulSoup para poder acceder al contenido solicitado
sopa_peliculas = BeautifulSoup(res_moviles.content, 'html.parser')
# sacamos info
puntuacion = sopa_peliculas.find("span", {"class": "sc-eb51e184-1 cxhhrI"}).text
argumento = sopa_peliculas.find("span", {"class": "sc-2d37a7c7-0 caYjFF"}).text
li = sopa_peliculas.find_all("li", {"class": "ipc-metadata-list__item"})
director = li[0].text.replace("Director","")
guionista = li[1].text.replace("Writer","")
duracion = li[6].text.replace("Writer","")
lista_actores = []
actores = sopa_peliculas.find_all("a", {"class": "sc-bfec09a1-1 gCQkeh"})
for i in range(0,11):   
    lista_actores.append(actores[i].text)

In [8]:
lista_actores


['Jon Foo',
 'Kelly Overton',
 'Cary-Hiroyuki Tagawa',
 'Ian Anthony Dale',
 'Cung Le',
 'Darrin Dewitt Henson',
 'Luke Goss',
 'Mircea Monroe',
 'Tamlyn Tomita',
 'Candice Hillebrand',
 'Marian Zapico']

In [68]:
peliculas_accion_sele.info_peliculas

{'Puntuación IMDB': 'None',
 'Puntuación Rotten Tomatoes': [],
 'Director': [],
 'Guionista': [],
 'Argumento': [],
 'Duración': [],
 'Nombre de la película': ['The Courier'],
 'Id pelicula': ['tt0995845']}

In [3]:
peliculas_accion_sele.info_actores

NameError: name 'peliculas_accion_sele' is not defined

In [79]:
class ExtraerDatosPeliculasSelenium:
    def __init__(self, archivo_csv): 
        self.archivo_csv = pd.read_csv(archivo_csv)  
        self.driver1 = webdriver.Chrome()        
        self.driver2 = webdriver.Chrome()        
        self.peliculas_selenium = []            
        self.info_peliculas_selenium = {"Puntuación IMDB": [],
                                        "Puntuación Rotten Tomatoes":[],
                                        "Director":[],
                                        "Guionista":[],
                                        "Argumento": [],
                                        "Duración":[],
                                        "Nombre de la película":[],
                                        "No entro": [],
                                        "No entro Tomatoes": []} 
        
    def pag_principal_imdb(self):
        # Inicializar el driver de Chrome
        sleep(2)
        self.driver1.maximize_window()
        self.driver1.get("https://www.imdb.com/")
        # acepta cookies
        sleep(2)
        try:
            self.driver1.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
            sleep(2)
        except:
            pass
        self.escoger_pelicula_csv()
        # self.pag_principal_tomatoes()
    

    def escoger_pelicula_csv (self):        
        # Itera por cada pelicula del csv
        for peli in range(4, len(self.archivo_csv)):
            titulo_a_buscar = self.archivo_csv['Titulo'][peli]
            anho_a_buscar = self.archivo_csv['Año'][peli]
            print(titulo_a_buscar,anho_a_buscar )
            self.buscar_pelicula_imdb (titulo_a_buscar, anho_a_buscar)
            # self.buscar_pelicula_tomatoes (titulo_a_buscar, anho_a_buscar)
            break


    def buscar_pelicula_imdb(self, titulo_a_buscar, anho_a_buscar):
        # Ingresar el título y año de la película en el campo de búsqueda
        self.driver1.find_element("css selector", "#suggestion-search").send_keys(f"{titulo_a_buscar} {anho_a_buscar}", Keys.ENTER)
        sleep(3)
        # Pulsa coincidencias exactas
        self.driver1.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary.sc-e8e4ce7-1.ksYNgL > div > div > a").click()
        print("coincidencias")
        sleep(2)
        # Pulsa filtro peliculas
        self.driver1.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-1 > div.sc-f2e21260-0.CbXa > section:nth-child(1) > div.ipc-chip-list--base.ipc-chip-list.ipc-chip-list--wrap.sc-816974d9-0.hjjagY > div.ipc-chip-list__scroller > a.more-results-ft-chip.ipc-chip.ipc-chip--on-base").click()
        print("filtro peli")
        self.escogemos_pelicula_correcta_imdb(titulo_a_buscar, anho_a_buscar)

    def escogemos_pelicula_correcta_imdb(self, titulo_a_buscar, anho_a_buscar ):
        for i in range(1,6):
            opcion_titulo = self.driver1.find_element("xpath", f'//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{i}]/div[2]/div/a').text
            print(opcion_titulo)
            opcion_anho = self.driver1.find_element("xpath", f'//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{i}]/div[2]/div/ul[1]/li/span').text
            print(opcion_anho)
            if titulo_a_buscar in str(opcion_titulo) and opcion_anho == anho_a_buscar:
                print("entra")
                self.driver1.find_element("css selector", f"#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-e8e4ce7-0.bEmpTm > div.sc-e8e4ce7-2.iXsuvN > ul > li:nth-child({i}) > div.ipc-metadata-list-summary-item__c > div > a").click()

    def pag_principal_tomatoes(self):    
        # Inicializar el driver de Chrome
        sleep(2)
        self.driver2.maximize_window()
        self.driver2.get("https://www.rottentomatoes.com/")
        # acepta cookies
        sleep(2)
        try:
            self.driver2.find_element("css selector", "#onetrust-accept-btn-handler").click()
            sleep(2)
            # self.extraer_info_pag_tomatoes()
            # self.escoger_pelicula()
        except:
            pass

    def buscar_pelicula_tomatoes(self, titulo_a_buscar, anho_a_buscar):
        # Ingresar el título y año de la película en el campo de búsqueda
        self.driver1.find_element("css selector", "#header-main > search-results-nav > search-results-controls > input[type=text]").send_keys(f"{titulo_a_buscar} {anho_a_buscar}", Keys.ENTER)
        sleep(3)
        # Iterar sobre las opciones de búsqueda en la página
        for i in range(1,6):
            opcion_anho = self.driver.find_element("css selector", f"li > div.info-wrap > span.year").text 
            opcion_titulo = self.driver.find_element("css selector", f"#search-results > search-page-result:nth-child(2) > ul > search-page-media-row:nth-child({i}) > a:nth-child(2)").text
            # Buscar la adecuada comparando titulo y año
            if  str(anho_a_buscar) == opcion_anho and str(titulo_a_buscar) == opcion_titulo:
                print("entra")
                puntuacion = self.driver.find_element("css selector", f'//*[@id="search-results"]/search-page-result[{i}]/ul/search-page-media-row[2]//li/div[1]/score-icon-critic-deprecated//div/span[2]').text


In [80]:
csv = "accion"
peliculas_accion_sele = ExtraerDatosPeliculasSelenium(csv)
peliculas_accion_sele.pag_principal_imdb()

Scott Pilgrim vs. the World 2010
coincidencias
filtro peli
Scott Pilgrim contra el mundo
2010
Halfaouine. El niño de las terrazas
1990


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[3]/div[2]/div/a"}
  (Session info: chrome=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF76456EEA2+31554]
	(No symbol) [0x00007FF7644E7ED9]
	(No symbol) [0x00007FF7643A872A]
	(No symbol) [0x00007FF7643F8434]
	(No symbol) [0x00007FF7643F853C]
	(No symbol) [0x00007FF76443F6A7]
	(No symbol) [0x00007FF76441D06F]
	(No symbol) [0x00007FF76443C977]
	(No symbol) [0x00007FF76441CDD3]
	(No symbol) [0x00007FF7643EA33B]
	(No symbol) [0x00007FF7643EAED1]
	GetHandleVerifier [0x00007FF764878B1D+3217341]
	GetHandleVerifier [0x00007FF7648C5AE3+3532675]
	GetHandleVerifier [0x00007FF7648BB0E0+3489152]
	GetHandleVerifier [0x00007FF76461E776+750614]
	(No symbol) [0x00007

In [ ]:
df_p.merge(df, on="actor", how = "inner") 

In [5]:
# def pag_principal_imdb(self)
driver1 = webdriver.Chrome()        
# Inicializar el driver de Chrome
sleep(2)
driver1.maximize_window()
driver1.get("https://www.imdb.com/")
# acepta cookies
sleep(2)
driver1.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
sleep(2)
        

In [50]:
csv = "accion"
archivo_csv = pd.read_csv(csv)
# def escoger_pelicula (self):        
        # Itera por cada pelicula del csv
for peli in range(4, len(archivo_csv)):
    titulo_a_buscar = archivo_csv['Titulo'][peli]
    anho_a_buscar = archivo_csv['Año'][peli]
#     buscar_pelicula_imdb (titulo_a_buscar, anho_a_buscar)
#     buscar_pelicula_tomatoes (titulo_a_buscar, anho_a_buscar)
    print(titulo_a_buscar,anho_a_buscar )
    break

#  def buscar_pelicula_imdb(self, titulo_a_buscar, anho_a_buscar):
# Ingresar el título y año de la película en el campo de búsqueda
driver1.find_element("css selector", "#suggestion-search").send_keys(f"{titulo_a_buscar} {anho_a_buscar}", Keys.ENTER)
sleep(3)
# Pulsa coincidencias exactas
driver1.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary.sc-e8e4ce7-1.ksYNgL > div > div > a").click()
sleep(2)
# Pulsa filtro peliculas
driver1.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-1 > div.sc-f2e21260-0.CbXa > section:nth-child(1) > div.ipc-chip-list--base.ipc-chip-list.ipc-chip-list--wrap.sc-816974d9-0.hjjagY > div.ipc-chip-list__scroller > a.more-results-ft-chip.ipc-chip.ipc-chip--on-base").click()
   

Scott Pilgrim vs. the World 2010
